In [1]:
import numpy as np
import glob
import datetime
from pmap import getmap
import pandas
import os

In [2]:
PATH='/mnt/ECMWF/processed/2016/FIX_MED_SEA/'  # NEW ECMWF

In [57]:
PATH='/mnt/oldECMWF/processed/2015/FIX_MED_SEA/'  # OLD ECMWF

In [3]:
folders=glob.glob(PATH+'calc_*')

In [4]:
filename=folders[0]+'/bathymetry.tif'
grid = getmap(filename)

gt=grid.GeoTr

width=grid.NCOLS
height=grid.NROWS

minx = gt[0]
miny = gt[3] + width*gt[4] + height*gt[5]
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3]

lon=np.linspace(minx,maxx,width,endpoint=True)
lat=np.linspace(miny,maxy,height,endpoint=True)


In [5]:
odate=datetime.datetime.strptime(folders[0][-11:],'%Y%m%d.%H')
print odate

2016-01-01 00:00:00


In [61]:
#print lon
#print lat

In [15]:
SAVEPATH='mnt/'#pandora/Users_Critech/Thomas/HYFLUX/'

In [7]:
OFILE='/mnt/pandora/Projects_Critech/EX_2015_CoastAlRisk/maps/data/s_b_data.txt'

In [8]:
obs=pandas.read_csv(OFILE,delimiter='\t')

In [9]:
obs

,ID,NAME,latobs,lonobs,latcalc,loncalc
0,1630,Almeira,36.8300,-2.4800,36.72506,-2.468386
1,1706,Valencia,39.4400,-0.3100,39.44040,-0.299000
2,819,Barcelona,41.3400,2.1600,41.27749,2.213909
3,1461,Palma de Mallorca,39.5600,2.6380,39.39457,2.614552
4,1741,Sete,43.3976,3.6991,43.31857,3.761195
5,1500,Corsica,42.9670,9.3500,43.09528,9.267036
6,1841,Carloforte,39.1480,8.3095,39.22599,8.231240
7,1848,Imperia,43.8783,8.0189,43.83057,8.149480
8,1846,Genova,44.4101,8.9255,44.32259,8.919400
9,1843,Civitavecchia,42.0940,11.7896,41.99920,11.677270


In [10]:
#SELECT POINT
point=21

In [11]:
plat,plon=obs['latcalc'][point],obs['loncalc'][point]

In [12]:
i=np.abs(lat-plat).argmin()
j=np.abs(lon-plon).argmin()


In [13]:
print i,j

176 471


In [16]:
SFOLDER=SAVEPATH+'{}/'.format(obs['ID'][point])
if not os.path.isdir(SFOLDER): os.makedirs(SFOLDER)

In [17]:
for ifolder in folders:
    rstamp=ifolder.split('/')[-1].split('_')[1] # the time stamp on the run folder (actual date of the run)
    rdate=datetime.datetime.strptime(rstamp,'%Y%m%d.%H')
    hfiles=glob.glob(ifolder+'/TIF_H*.tif')
    t=[]
    hobs=[]
    for ifile in hfiles:  
        dat=getmap(ifile)
        hobs.append(np.flipud(dat.data)[i,j])
        ti=ifile.split('/')[-1].split('_H_')[1].split('.')[0]
        if ti == 0: # there is a new restart
            t.append(rdate)
            odate=rdate # set new odate
        else:
            t.append(odate+datetime.timedelta(hours=float(ti)/60./60.))
    np.savetxt(SFOLDER+rstamp,np.transpose([t, hobs]),delimiter=",", fmt="%s")


ValueError: invalid literal for float(): 00-10799